
'''<br>
@Author: Rahul<br> 
@Date: 2024-10-24<br>
@Last Modified by: Rahul <br>
@Last Modified time: 2024-10-24<br>
@Title: Python program to Import and Export data Using Boto3<br>
'''

Import from Database and Export to S3 bucket

In [8]:
import pymssql
import csv
import boto3
from botocore.exceptions import NoCredentialsError
import pandas as pd

In [3]:

def connect_to_database(endpoint, port, username, password, database):
    conn = pymssql.connect(server=f"{endpoint}:{port}", user=username, password=password, database=database)
    return conn

def export_data_to_csv(conn, table_name, csv_file):
    cursor = conn.cursor()
    query = f"SELECT * FROM {table_name}"
    cursor.execute(query)
    
    with open(csv_file, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([i[0] for i in cursor.description])  # Write headers
        writer.writerows(cursor.fetchall())  # Write data
    
    print(f"Data exported to {csv_file}")

def upload_file_to_s3(file_name, bucket_name, s3_file_name):
    s3 = boto3.client('s3')

    try:
        s3.upload_file(file_name, bucket_name, s3_file_name)
        print(f"File uploaded to S3: {bucket_name}/{s3_file_name}")
    except FileNotFoundError:
        print("The file was not found.")
    except NoCredentialsError:
        print("AWS credentials not available.")

if __name__ == "__main__":
    server = 'rahulbldb-rj.cxsygi2uk94v.ap-south-1.rds.amazonaws.com'
    port = 1433  
    database = 'rdsrahul'
    username = 'admin'
    password = 'RaHuL637j'

    bucket_name = 'rahulbllamdabucket'
    file_name = 'order_data.csv'
    local_file_path = 'order_data.csv'

    table_name = "Orders"

    conn = connect_to_database(server, port, username, password, database)
    try:
        export_data_to_csv(conn, table_name, local_file_path)
        
        upload_file_to_s3(local_file_path, bucket_name, file_name)
    finally:
        conn.close()


Data exported to order_data.csv
File uploaded to S3: rahulbllamdabucket/order_data.csv


In [10]:

def connect_to_database(endpoint, port, username, password, database):
    try:
        conn = pymssql.connect(server=f"{endpoint}:{port}", user=username, password=password, database=database)
        print("Database connection successful!")
        return conn
    except Exception as e:
        print(f"Error connecting to database: {e}")
        return None

def download_file_from_s3(bucket_name, s3_file_name, local_file_name):
    s3 = boto3.client('s3')

    try:
        s3.download_file(bucket_name, s3_file_name, local_file_name)
        print(f"File downloaded from S3: {bucket_name}/{s3_file_name} to {local_file_name}")
    except NoCredentialsError:
        print("AWS credentials not available.")
    except Exception as e:
        print(f"Error downloading file from S3: {e}")

def import_data_from_csv(conn, table_name, csv_file, identity_column):
    cursor = conn.cursor()

    df = pd.read_csv(csv_file)

    if identity_column in df.columns:
        df = df.drop(columns=[identity_column])

    for col in df.select_dtypes(include=['object']):
        try:
            df[col] = pd.to_datetime(df[col], errors='coerce').dt.strftime('%Y-%m-%d')
        except Exception as e:
            print(f"Error converting column '{col}': {e}")

    
    df.dropna(inplace=True)  

    for index, row in df.iterrows():
        placeholders = ', '.join(['%s'] * len(row)) 
        insert_query = f"INSERT INTO {table_name} ({', '.join(df.columns)}) VALUES ({placeholders})"
        cursor.execute(insert_query, tuple(row))

    conn.commit()
    print(f"Data imported from {csv_file} to {table_name}")
    cursor.close()

if __name__ == "__main__":

    server = 'rahulbldb-rj.cxsygi2uk94v.ap-south-1.rds.amazonaws.com'
    port = 1433  
    database = 'rdsrahul'
    username = 'admin'
    password = 'RaHuL637j'

 
    bucket_name = 'rahulbllamdabucket'
    s3_file_name = 'exported_data.csv'  
    local_file_path = 'imported_data.csv'  

    table_name = "Orders"
    identity_column = 'OrderID'  

    conn = connect_to_database(server, port, username, password, database)
    if conn:
        try:
            download_file_from_s3(bucket_name, s3_file_name, local_file_path)

            import_data_from_csv(conn, table_name, local_file_path, identity_column)
        finally:
            conn.close()


Database connection successful!
File downloaded from S3: rahulbllamdabucket/exported_data.csv to imported_data.csv
Data imported from imported_data.csv to Orders
